In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, BertForPreTraining, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
import torch


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
config = BertConfig.from_pretrained("bert-base-cased", hidden_dropout_prob=0.5, attention_probs_dropout_prob=0.5, num_labels = 1)
model = BertForSequenceClassification(config)




In [4]:
dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/train.jsonl', 'test': '/content/drive/MyDrive/test.jsonl'})


Using custom data configuration default-c36ab8dfd69bb5fd
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-c36ab8dfd69bb5fd/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def preprocess_data(examples):
    # encode a batch of sentences
    encoding = tokenizer(examples['sentence1'], examples['sentence2'], truncation=True)
    # add labels as a list
    encoding["labels"] = float(examples["label"])
    #encoding['position_ids'] = examples['idx']
    
    return encoding
data_collator = DataCollatorWithPadding(tokenizer)

# tokenize sentences + add labels
encoded_dataset = dataset.map(preprocess_data)
encoded_dataset = encoded_dataset.remove_columns(['word','sentence1','sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'])
for k,v in encoded_dataset.items():
     print(k, v.shape)
# turn into PyTorch dataset
encoded_dataset.set_format("torch")
print(tokenizer.decode(encoded_dataset['train']['input_ids'][2]))


small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(600))
small_eval_dataset = encoded_dataset["test"].shuffle(seed=42).select(range(600))


Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-c36ab8dfd69bb5fd/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-0e6111c9aed37798.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-c36ab8dfd69bb5fd/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-fe4729bfa34919b6.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/json/default-c36ab8dfd69bb5fd/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-28672534af4ecc44.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/json/default-c36ab8dfd69bb5fd/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-0bf96aaa8bdf04e2.arrow


train (5000, 4)
test (638, 4)
[CLS] Run rogue. [SEP] She ran 10 miles that day. [SEP]


In [17]:
import numpy as np
from datasets import load_metric

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/",
    save_strategy = "epoch",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay = .00005,
    learning_rate = .00002,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset = small_train_dataset, 
    eval_dataset= small_eval_dataset, 
    data_collator = data_collator,  
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 600
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.292808,0.495000
2,No log,0.249983,0.495000
3,No log,0.260789,0.495000
4,No log,0.331679,0.495000
5,No log,0.460216,0.500000


***** Running Evaluation *****
  Num examples = 600
  Batch size = 8


[0.71181035 0.71196043 0.71139836 0.7106704  0.71217597 0.712851
 0.712562   0.71185577 0.71270484 0.7109373  0.7133547  0.7127246
 0.71050286 0.71183807 0.71071726 0.71188885 0.7122783  0.712643
 0.71236    0.71211743 0.7118545  0.71064794 0.71208364 0.71343553
 0.712055   0.7126492  0.71290755 0.71162224 0.71108234 0.7110768
 0.71196586 0.7109225  0.71247894 0.7116919  0.71284074 0.71156204
 0.712222   0.7131896  0.7117051  0.71228755 0.7121426  0.71376026
 0.71155924 0.7116823  0.71265906 0.71291286 0.7114619  0.7116993
 0.71188724 0.711385   0.71191657 0.71155035 0.7122624  0.71253574
 0.7116257  0.7121468  0.71137685 0.71206605 0.71180034 0.71315783
 0.710918   0.71139985 0.71155274 0.71100605 0.71172094 0.7115129
 0.7128883  0.7131497  0.71113974 0.7113478  0.7129093  0.7115997
 0.7115689  0.71220666 0.7118974  0.7123105  0.7118417  0.71055514
 0.7116182  0.7116751  0.7120293  0.7126617  0.71220803 0.71225613
 0.7120724  0.7121313  0.7128567  0.7123034  0.71160066 0.7119371
 0.71

Saving model checkpoint to /content/drive/MyDrive/checkpoint-75
Configuration saved in /content/drive/MyDrive/checkpoint-75/config.json
Model weights saved in /content/drive/MyDrive/checkpoint-75/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8


...

[0.7907864  0.7693038  0.77910453 0.8354303  0.8004541  0.7856768
 0.7867111  0.7378673  0.8433913  0.8454407  0.76291174 0.74260694
 0.8425393  0.7753131  0.869392   0.7789111  0.7849271  0.77446514
 0.8129709  0.790763   0.7767129  0.8748028  0.74664766 0.82341105
 0.78672063 0.775474   0.80844927 0.76281613 0.84732693 0.8441023
 0.76572645 0.8069097  0.8030061  0.808015   0.78787607 0.786453
 0.7842104  0.78178    0.7952957  0.82008207 0.7448947  0.77633333
 0.8178788  0.81385034 0.7941089  0.7716964  0.7237737  0.7516978
 0.7883213  0.7622635  0.7622427  0.7610239  0.7657693  0.8082181
 0.82380193 0.75601596 0.7960599  0.69971216 0.75414383 0.7788166
 0.8132168  0.79911846 0.8509171  0.8092199  0.84617555 0.8345424
 0.78822243 0.75595456 0.8081868  0.80339974 0.73474157 0.8029581
 0.8185637  0.7575072  0.7951936  0.79077715 0.78127855 0.84909916
 0.7689476  0.7844475  0.7338511  0.7895025  0.76767963 0.79228514
 0.73417705 0.7573265  0.8133088  0.781023   0.82817364 0.8280764
 0.84

Saving model checkpoint to /content/drive/MyDrive/checkpoint-300
Configuration saved in /content/drive/MyDrive/checkpoint-300/config.json
Model weights saved in /content/drive/MyDrive/checkpoint-300/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8


[0.95884913 0.9362701  0.946042   1.0010352  0.98062605 0.9752625
 0.9806158  0.92177624 1.0212125  0.9908372  0.9544889  0.94069326
 0.9981889  0.971331   1.0087141  0.9535654  0.96500754 0.97795594
 0.970015   0.97175145 0.956248   1.0230052  0.93130714 0.9951747
 0.97432643 0.96753496 0.9870624  0.94235504 0.99520916 0.99789065
 0.95395017 0.9699305  0.9819696  0.9697532  0.98220044 0.93966967
 0.971958   0.9580221  0.9714167  0.99540454 0.94634134 0.96172225
 0.99348825 0.9873543  0.97909075 0.9726418  0.8975039  0.9287923
 0.97484607 0.9178661  0.94436276 0.9451091  0.94250536 0.9777749
 0.9990448  0.93908507 0.9604432  0.8764992  0.91816187 0.9639073
 0.9849275  0.9587651  0.9978423  0.981145   1.0065327  0.9895745
 0.9603948  0.9403666  0.98132783 0.9747945  0.915248   0.98932856
 0.98840344 0.9464731  0.9491453  0.9758887  0.96437544 1.0149089
 0.937961   0.9567885  0.92662615 0.96633875 0.94185674 0.9567555
 0.92466617 0.93250805 0.9899011  0.98614335 0.9760566  1.005091
 0.99

Saving model checkpoint to /content/drive/MyDrive/checkpoint-375
Configuration saved in /content/drive/MyDrive/checkpoint-375/config.json
Model weights saved in /content/drive/MyDrive/checkpoint-375/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/MyDrive/checkpoint-150 (score: 0.2499827891588211).


TrainOutput(global_step=375, training_loss=0.10179373168945313, metrics={'train_runtime': 72.2998, 'train_samples_per_second': 41.494, 'train_steps_per_second': 5.187, 'total_flos': 47454119425104.0, 'train_loss': 0.10179373168945313, 'epoch': 5.0})

In [7]:
def model_init():
    return BertForSequenceClassification.from_pretrained( "distilbert-base-uncased", num_labels=1)
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")


loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Model config BertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dim": 768,
  "dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "n_heads":

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.263877,0.495000


***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/run-0/checkpoint-38
Configuration saved in /content/drive/MyDrive/run-0/checkpoint-38/config.json
Model weights saved in /content/drive/MyDrive/run-0/checkpoint-38/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/run-0/checkpoint-38/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/run-0/checkpoint-38/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/MyDrive/run-0/checkpoint-38 (score: 0.26387715339660645).
[I 2021-11-27 22:48:39,548] Trial 0 finished with value: 0.495 and parameters: {'learning_rate': 1.936354435181433e-06, 'num_train_epochs': 1, 'seed': 5, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.495.
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/confi

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.250851,0.495000
2,No log,0.250924,0.495000
3,No log,0.252531,0.495000


***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/run-1/checkpoint-38
Configuration saved in /content/drive/MyDrive/run-1/checkpoint-38/config.json
Model weights saved in /content/drive/MyDrive/run-1/checkpoint-38/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/run-1/checkpoint-38/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/run-1/checkpoint-38/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/run-1/checkpoint-76
Configuration saved in /content/drive/MyDrive/run-1/checkpoint-76/config.json
Model weights saved in /content/drive/MyDrive/run-1/checkpoint-76/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/run-1/checkpoint-76/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/run-1/checkpoint-76/special_tokens_map.json
***** Running 

...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.274658,0.495000
2,No log,0.250933,0.495000


***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/run-3/checkpoint-150
Configuration saved in /content/drive/MyDrive/run-3/checkpoint-150/config.json
Model weights saved in /content/drive/MyDrive/run-3/checkpoint-150/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/run-3/checkpoint-150/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/run-3/checkpoint-150/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/run-3/checkpoint-300
Configuration saved in /content/drive/MyDrive/run-3/checkpoint-300/config.json
Model weights saved in /content/drive/MyDrive/run-3/checkpoint-300/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/run-3/checkpoint-300/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/run-3/checkpoint-300/special_tokens_map.json


Tr

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.252458,0.495000
2,No log,0.270648,0.495000
3,No log,0.250123,0.495000
4,No log,0.251451,0.495000
5,No log,0.249977,0.495000


***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/run-4/checkpoint-19
Configuration saved in /content/drive/MyDrive/run-4/checkpoint-19/config.json
Model weights saved in /content/drive/MyDrive/run-4/checkpoint-19/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/run-4/checkpoint-19/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/run-4/checkpoint-19/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 600
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/run-4/checkpoint-38
Configuration saved in /content/drive/MyDrive/run-4/checkpoint-38/config.json
Model weights saved in /content/drive/MyDrive/run-4/checkpoint-38/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/run-4/checkpoint-38/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/run-4/checkpoint-38/special_tokens_map.json
***** Running 

KeyboardInterrupt: ignored

In [ ]:

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/",
    save_strategy = "epoch",
    evaluation_strategy="epoch",
    per_device_train_batch_size=13,
    per_device_eval_batch_size=13,
    num_train_epochs=15,
    weight_decay = .00005
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics,
   

)
trainer.evaluate()
